In [15]:
import shelve
import pandas as pd
from read_tweets import parse_raw_tweets
from tokenizer import tokenize

In [9]:
batch_path = '../private_data/batch.txt'
parse_raw_tweets(batch_path)
shelf = shelve.open('../private_data/tweet_processing')
users_df = shelf['users_df']
tweets_df = shelf['tweets_df']
shelf.close()
tweets_df.head()

,author_id,tweet_id,tweet_text
0,2743106511,1271114971078897671,RT @RepJerryNadler: When we impeached Presiden...
1,48916193,1271114970902736898,@GatewayPeople Shame on YOU for hosting Trump'...
2,321951940,1271114970844053504,RT @keithboykin: General Mark Milley apologize...
3,1018650672353312769,1271114970810298368,"@ddale8 ""Don't ever, for any reason, do anythi..."
4,1266178928470867969,1271114970747613187,RT @PressSec: America's great military bases a...


In [17]:
merged = pd.merge(tweets_df, users_df, on='author_id')
merged.head()

,author_id,tweet_id,tweet_text,location
0,2743106511,1271114971078897671,RT @RepJerryNadler: When we impeached Presiden...,"North Carolina, USA"
1,1018650672353312769,1271114970810298368,"@ddale8 ""Don't ever, for any reason, do anythi...","Los Angeles, CA"
2,292404883,1271114970588229644,RT @VANDERBUSHJANE: President Trump has banned...,"Olmsted Falls, OH"
3,1209320835535822848,1271114970571444228,RT @petermaer: ⁦@JoeBiden⁩ :”It's my greatest ...,earth usa🐈🐾🌊
4,1026847067849871360,1271114970571431937,This has Stephen Miller written all over it. T...,"Wisconsin, USA"


# Combining with Location data

In [27]:
import shelve
shelf = shelve.open('../public_data/location_data')
state_strings = shelf['state_strings']
foreign_entities = shelf['foreign_entities_df']
foreign_entities

,name,pop,in_us,state,raw_name_string
4,adrar,200834,False,None,None
5,ain defla,450280,False,None,None
6,ain temouchent,299341,False,None,None
7,algiers (el djazair),2712944,False,None,None
8,annaba,442230,False,None,None
...,...,...,...,...,...
9928,"garth, menai (bangor)",5125,False,None,None
9929,"haverfordwest: prendergast, rudbaxton",2952,False,None,None
9930,"brackla, coity",13820,False,None,None
9931,"crickhowell, llangattock",3800,False,None,None


## Tokenization